In [7]:
from openpyxl import load_workbook
import pandas as pd

In [8]:
df = pd.read_excel('kotlin-code-code.xlsx', sheet_name = 'data')

In [9]:
df.dtypes



category       object
resource-id    object
code           object
dtype: object

In [10]:
df['category'] = pd.Series(df['category'], dtype="string")
df['resource-id'] = pd.Series(df['resource-id'], dtype="string")
df['code'] = pd.Series(df['code'], dtype="string")
df = df.drop_duplicates()
df = df.sort_values(by=['resource-id'])

In [11]:
# final_data = []
# for index, row in df.iterrows():
#   final_data.append( row['category']+' '+row['resource-id'])


In [12]:
# final_data = pd.DataFrame(final_data)
# final_data.columns = ['context']
# final_data['context'] = pd.Series(final_data['context'], dtype="string")
# final_data = final_data.join(df['code'])

In [13]:
# final_data.drop_duplicates()

In [14]:
# final_data.dtypes
# len(final_data)
# final_data
len(df['code'].unique())

737

In [15]:
df = df.reset_index(drop=True)

In [16]:
df

,category,resource-id,code
0,Science & Education,account_email,val accountEmail: TextView
1,Science & Education,account_email,override fun bindView(holder: AccountViewHolde...
2,Internet,addBookmarksPopupMenuItem,private fun createPopupMenu()
3,Internet,addBookmarksPopupMenuItem,private fun renderPopupMenus(browserShowing: B...
4,Internet,addFavoritePopupMenuItem,private fun createPopupMenu()
...,...,...,...
958,Time,week_day_name,private fun create4x2RemoteViews(context: Cont...
959,Time,week_day_name,private fun create2x2RemoteViews(context: Cont...
960,Time,week_day_name,private fun create4x1RemoteViews(context: Cont...
961,Internet,whitelistPopupMenuItem,private fun createPopupMenu()


In [17]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

In [18]:
# X = df[['category', 'resource-id']]
X = df[[ 'resource-id']]
y = df['code']

In [19]:

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42 )

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

print(len(X_train), len(X_test), type(X_train))

770 193 <class 'pandas.core.frame.DataFrame'>


In [20]:
train_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train_enc.fit(X_train)
encoded_train_X = train_enc.transform(X_train)

test_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
test_enc.fit( np.array(y_train).reshape(1, -1).transpose() )
encoded_train_y = test_enc.transform(  np.array(y_train).reshape(1, -1).transpose() )


In [21]:
len(y_train.unique())

609

In [22]:

from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import tensorflow as tf

In [23]:
encoded_test_X = train_enc.transform(X_test)
encoded_test_y = test_enc.transform( np.array(y_test).reshape(1, -1).transpose() )

In [24]:
#Configure the model
def config_model(epochs, batch_size):
  model = tf.keras.models.Sequential([tf.keras.layers.Dense(500, activation=tf.nn.relu), 
                                      # tf.keras.layers.Dense(1000, activation=tf.nn.relu),
                                      # tf.keras.layers.Dense(5000, activation=tf.nn.relu),
                                      # tf.keras.layers.Dense(5000, activation=tf.nn.relu),
                                      tf.keras.layers.Dense(750, activation=tf.nn.softmax)])
  model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
  model.fit(encoded_train_X, encoded_train_y, shuffle=True , 
            epochs=epochs, batch_size=batch_size, validation_split=0.20)
  
  return model

In [25]:
encoded_train_y.shape

(770, 1)

In [26]:
import re

def bleu_score( expected, actual ):

  expected_words_set = set( re.split('\s+', re.sub("[^0-9a-zA-Z]+", " ", expected)))
  actual_words_set = set( re.split('\s+', re.sub("[^0-9a-zA-Z]+", " ", actual)))

  total = max( len(expected_words_set), len(actual_words_set))
  return len( expected_words_set.intersection(actual_words_set) ) / total

In [ ]:
epochs = [100, 200, 300, 400, 500]
batches = [16, 32, 64, 128, 256]
models_data = []
for i in range(len(epochs)):
  for j in range(len(batches)):
    model = config_model(epochs[i], batches[j])
    models_data.append([ epochs[i], batches[j], model])
    # break


Replace extra characters

In [28]:


# print( encoded_test_X.shape, encoded_test_y.shape)

def getResult(model_data):
  temp = []
  counter = 0

  blue_scores = []

  for index, data in enumerate(encoded_test_X):
    # print( data )
    
    result = model_data[2].predict( pd.DataFrame(data).transpose() ).transpose()
    k = 5
    top_k_index = np.argsort(result, axis=0, kind='heap', order=None)[-k:][::-1]

    max_score = 0.0
    for idx in top_k_index:
      idx = encoded_train_y[idx]
      expected:str = test_enc.inverse_transform( np.array(idx).reshape(-1, 1) ).flatten()[0]
      actual:str = y_test[index]

      max_score = max( max_score,  bleu_score(expected, actual) ) 
      if test_enc.inverse_transform( np.array(idx).reshape(-1, 1) ).flatten()[0] == y_test[index]:
        counter = counter + 1

    blue_scores.append(max_score)
  accuracy = ( counter / len(X_test))
  score = sum(blue_scores) / len(blue_scores)

  return [accuracy, score]

In [29]:
# blue_scores

In [30]:
for model_data in models_data:
  result = getResult(model_data)
  model_data.extend( result )


print(models_data)

[[100, 16, <keras.engine.sequential.Sequential object at 0x7f3181764c90>, 0.0, 0.39910309525658666], [100, 32, <keras.engine.sequential.Sequential object at 0x7f317611eed0>, 0.0, 0.3940001353799297], [100, 64, <keras.engine.sequential.Sequential object at 0x7f3176032b90>, 0.0, 0.40244351055600847], [100, 128, <keras.engine.sequential.Sequential object at 0x7f31706c3bd0>, 0.0, 0.3944012844019261], [100, 256, <keras.engine.sequential.Sequential object at 0x7f317178bc90>, 0.0, 0.38648533736680807], [200, 16, <keras.engine.sequential.Sequential object at 0x7f3176186210>, 0.0051813471502590676, 0.3924029679742398], [200, 32, <keras.engine.sequential.Sequential object at 0x7f3170ceb9d0>, 0.0051813471502590676, 0.39400611673306146], [200, 64, <keras.engine.sequential.Sequential object at 0x7f3170cabc10>, 0.0051813471502590676, 0.38749466618016], [200, 128, <keras.engine.sequential.Sequential object at 0x7f317610b610>, 0.0, 0.3908492182622343], [200, 256, <keras.engine.sequential.Sequential ob

In [31]:
for model in models_data:
  model.pop(2)
  print(model)

[100, 16, 0.0, 0.39910309525658666]
[100, 32, 0.0, 0.3940001353799297]
[100, 64, 0.0, 0.40244351055600847]
[100, 128, 0.0, 0.3944012844019261]
[100, 256, 0.0, 0.38648533736680807]
[200, 16, 0.0051813471502590676, 0.3924029679742398]
[200, 32, 0.0051813471502590676, 0.39400611673306146]
[200, 64, 0.0051813471502590676, 0.38749466618016]
[200, 128, 0.0, 0.3908492182622343]
[200, 256, 0.0, 0.3900956202059846]
[300, 16, 0.0, 0.3932807070186962]
[300, 32, 0.0051813471502590676, 0.37872311012339743]
[300, 64, 0.0051813471502590676, 0.38583829865591707]
[300, 128, 0.0, 0.38912688542895957]
[300, 256, 0.0, 0.38678655138936846]
[400, 16, 0.0051813471502590676, 0.3829985985957772]
[400, 32, 0.0051813471502590676, 0.40178285118831997]
[400, 64, 0.0051813471502590676, 0.38897358026955886]
[400, 128, 0.0051813471502590676, 0.38759497183136493]
[400, 256, 0.0, 0.38770358726686227]
[500, 16, 0.0051813471502590676, 0.38466582888995027]
[500, 32, 0.0, 0.3745358609879617]
[500, 64, 0.0051813471502590676

Decode Values

In [32]:
result_df = pd.DataFrame(models_data)

In [33]:
result_df.columns = ['Epochs', 'Batch Size', 'Accuracy', 'Bleu Score']

In [34]:
result_df

,Epochs,Batch Size,Accuracy,Bleu Score
0,100,16,0.000000,0.399103
1,100,32,0.000000,0.394000
2,100,64,0.000000,0.402444
3,100,128,0.000000,0.394401
4,100,256,0.000000,0.386485
5,200,16,0.005181,0.392403
6,200,32,0.005181,0.394006
7,200,64,0.005181,0.387495
8,200,128,0.000000,0.390849
9,200,256,0.000000,0.390096


In [35]:
result_df.to_excel(excel_writer='result_nmt_without_category.xlsx', sheet_name='data')

In [ ]:
!pip install hvplot
import pandas as pd

In [4]:
result_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/result_nmt.xlsx', sheet_name = 'data')

In [6]:
import numpy as np
import hvplot.pandas
import pandas as pd

result_df.plot(x='Epochs', y=['Accuracy', 'Bleu Score',  ], kind='scatter', backend='plotly', 
    width=500,   height = 400
)


Output hidden; open in https://colab.research.google.com to view.

In [5]:
result_df.plot(x='Batch Size', y=['Accuracy', 'Bleu Score',  ], kind='scatter', backend='plotly', 
    width=500,   height = 400
)